# Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# global imports
import os
from google.colab import drive


In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import json
import re
import nltk
import string

In [4]:
! pip install contractions
!pip install -U spacy
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [6]:
import spacy
# import ktrain
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
import re
from sklearn.feature_extraction.text import CountVectorizer
#from ktrain.text import StandardTextPreprocessor
import contractions
from gensim import corpora
from gensim.models import LdaModel
from gensim.parsing.preprocessing import preprocess_string
# getting stop_words from nltk and initializing a lemmatizer
stop_words = set(nltk.corpus.stopwords.words('english'))
lemmatizer = nltk.stem.WordNetLemmatizer()
from sklearn.cluster import KMeans
import time
import pickle
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize


# Data Cleaning

In [7]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
# import cupy as cp
import pandas as pd
import xml.etree.ElementTree as ET
from google.colab import drive
from sklearn.compose import make_column_transformer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [8]:
sentiment_words = [
    "couldn't", "shouldn't", "wasn't", "doesn't", "wouldn't",
    "nor", "won't", "mustn't", "isn't", "weren't", "haven't",
    "hadn't", "doesn't", "wasn't", "aren't", "hadn't", "shan't",
    "about", "don't", "shouldn't", "didn't", "wouldn't", "needn't",
    "mightn't", "haven't", "not", "no", "very"
]

def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Convert to lowercase
    text = text.lower()

    # Tokenize text
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))

    # Remove sentiment affecting words
    for i in sentiment_words:
      if i in stop_words:
        stop_words.remove(i)

    filtered_tokens = [word for word in tokens if word not in stop_words]

    # Join tokens back into string
    clean_text = ' '.join(filtered_tokens)

    return clean_text


def clean_aspect(aspect):
    # Convert to lowercase
    aspect = aspect.lower()

    # Remove punctuation
    aspect = aspect.translate(str.maketrans('', '', string.punctuation))

    return aspect

In [25]:
# load the dataset with labels

drive_path = '/content/drive/MyDrive/'
dataset_path = drive_path + 'Restaurants_Train_v2.xml'
dataset = []

f = ET.parse(dataset_path)
for i in f.getroot():
  for j in i.find("aspectCategories"):
    dataset.append({
        "input_text": clean_text(i.find("text").text),
        "aspect": clean_aspect(j.attrib['category']),
        "polarity": j.attrib['polarity']
    })

bgw_vectorizer = CountVectorizer(binary=False, stop_words='english', ngram_range=(1,4))
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,1))
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

In [26]:
df = pd.DataFrame(train_dataset)

In [27]:
df.head()

,input_text,aspect,polarity
0,decor nice casual fine dining,ambience,conflict
1,find returning quite often,anecdotesmiscellaneous,positive
2,seating trying get maximum amount people resta...,ambience,negative
3,like took leftover chicken poured oil sprinkle...,food,negative
4,finest latin women youll see life hang jimmys,ambience,positive


In [28]:
df['input_text'][4]

'finest latin women youll see life hang jimmys'

# Getting aspect nouns



In [29]:
# Load spaCy model
nlp = spacy.load("en_core_web_lg")


# Define a function to extract nouns (aspects) from a review
def extract_nouns_from_review(review, nlp):
    # Process the review using spaCy pipeline
    doc = nlp(review)
    # Extract nouns from the review
    nouns = [token.text for token in doc if token.pos_ == "NOUN"]
    return list(set(nouns))

# Extract nouns (aspects) for each review and add them as a new column to the DataFrame
df['extracted_nouns'] = df['input_text'].apply(lambda review: extract_nouns_from_review(review, nlp))

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [30]:
df.head()

,input_text,aspect,polarity,extracted_nouns
0,decor nice casual fine dining,ambience,conflict,"[decor, dining]"
1,find returning quite often,anecdotesmiscellaneous,positive,[]
2,seating trying get maximum amount people resta...,ambience,negative,"[people, amount, neighbor, seating]"
3,like took leftover chicken poured oil sprinkle...,food,negative,"[chicken, pepper, oil, red, sauce, powder]"
4,finest latin women youll see life hang jimmys,ambience,positive,"[life, women]"


In [31]:
df.shape

(2970, 4)

In [32]:
df = df[~(df['input_text']=='')] #removing blank reviews

In [33]:
df = df[df['input_text'].apply(lambda x: len(nltk.word_tokenize(x)))>1] #removing single worded reviews

In [34]:
df.shape

(2914, 4)

In [35]:
df.to_csv(drive_path+'semeval_cleaned.csv', index=False)

In [37]:
df=pd.read_csv(drive_path+'semeval_cleaned.csv')

In [38]:
df.shape

(2914, 4)

# Getting aspect categories

# BERT Topic Modelling

In [39]:
df=pd.read_csv(drive_path+'semeval_cleaned.csv')

import warnings
warnings.filterwarnings("ignore")

In [40]:
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 22.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 19.3 MB/s eta 0:00:00
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 7.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (7

In [41]:
%%time
from bertopic import BERTopic
model = BERTopic(verbose=True,embedding_model='paraphrase-MiniLM-L3-v2', min_topic_size= 4)
headline_topics, _ = model.fit_transform(df.input_text)

2024-04-16 15:11:04,835 - BERTopic - Embedding - Transforming documents to embeddings.


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.04k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/92 [00:00<?, ?it/s]

2024-04-16 15:11:14,647 - BERTopic - Embedding - Completed ✓
2024-04-16 15:11:14,648 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-04-16 15:11:34,171 - BERTopic - Dimensionality - Completed ✓
2024-04-16 15:11:34,173 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-04-16 15:11:34,312 - BERTopic - Cluster - Completed ✓
2024-04-16 15:11:34,328 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-04-16 15:11:34,568 - BERTopic - Representation - Completed ✓


CPU times: user 48.9 s, sys: 1.76 s, total: 50.7 s
Wall time: 1min 7s


In [42]:
freq = model.get_topic_info()
print("Number of topics: {}".format( len(freq)))
freq.head()

Number of topics: 189


,Topic,Count,Name,Representation,Representative_Docs
0,-1,846,-1_very_not_first_place,"[very, not, first, place, table, get, one, lik...","[very fun experience group kids 6 would love, ..."
1,0,60,0_pizza_pizzas_soggy_crust,"[pizza, pizzas, soggy, crust, parked, fresca, ...","[pizza really good, pizza delicious, pizza great]"
2,1,59,1_restaurant_mermaid_inn_give,"[restaurant, mermaid, inn, give, based, friend...",[portions very small sauces overlyambitious us...
3,2,41,2_sushi_echoes_irritating_chill,"[sushi, echoes, irritating, chill, horrid, aco...",[20 gets unlimited sushi very high quality eve...
4,3,35,3_bagels_bagel_cream_freeze,"[bagels, bagel, cream, freeze, spread, capex, ...","[good bagels good cream cheese, pick bagel bes..."


In [43]:
model.visualize_barchart(top_n_topics=8)

In [44]:
model.visualize_topics()

In [45]:
model.visualize_hierarchy(top_n_topics=20)

We will take 4 main topics out of these clusters - food, service, ambience, price

# Using cosine similarity to get aspect of each review

In [46]:
import spacy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
# Load spaCy model with GloVe vectors
#!python -m spacy download en_core_web_lg #run this for every new session
nlp = spacy.load("en_core_web_lg")

# Define aspects and their corresponding words
aspects = {
    'food': 'food delicious tasty delicious flavor meal cuisine chef portion seasoned authentic',
    'service': 'service staff friendly helpful attentive prompt',
    'ambience': 'ambience ambiance atmosphere decor decoration cozy elegant modern chic view clean dirty dusty',
    'price': 'price value affordable expensive budget cost cheap',
    # 'anecdotesmiscellaneous': 'location parking cleanliness presentation variety'

}


# Define a function to compute aspect embeddings
def compute_aspect_embeddings(aspects, nlp):
    aspect_embeddings = {}
    for aspect, words in aspects.items():
        # Tokenize and average the word vectors for each aspect
        tokens = nlp(words)
        aspect_vectors = [token.vector for token in tokens if token.has_vector]
        aspect_embeddings[aspect] = np.mean(aspect_vectors, axis=0)
    return aspect_embeddings

# Define a function to compute review embeddings
def compute_review_embedding(review, nlp):
    # Process the review using spaCy pipeline
    doc = nlp(review)
    sentence_embeddings = []
    for sent in doc.sents:
        # Compute sentence embedding by averaging word vectors
        sent_vector = np.mean([word.vector for word in sent if word.has_vector], axis=0)
        sentence_embeddings.append(sent_vector)
    # Replace new words with zeros and compute review embedding
    review_embedding = np.mean(sentence_embeddings, axis=0)
    return np.nan_to_num(review_embedding)


def compute_sentence_embedding(sentence, nlp):
    # Process the sentence using spaCy pipeline
    doc = nlp(sentence)
    # Compute sentence embedding by averaging word vectors
    sent_vector = np.mean([word.vector for word in doc if word.has_vector], axis=0)
    # Replace NaN values with zeros
    return np.nan_to_num(sent_vector)


# Compute semantic similarity between review sentences and aspect embeddings
def compute_semantic_similarity(review_sentences, aspect_embeddings):
    aspect_names = list(aspect_embeddings.keys())
    similarity_scores = []
    for sentence in review_sentences:
        sentence_vector = compute_review_embedding(sentence, nlp)
        sentence_vector = sentence_vector.reshape(1, -1)  # Reshape to match cosine_similarity function input
        sentence_similarity = []
        for aspect_name, aspect_vector in aspect_embeddings.items():
            aspect_vector = aspect_vector.reshape(1, -1)  # Reshape to match cosine_similarity function input
            similarity = cosine_similarity(sentence_vector, aspect_vector)[0][0]
            sentence_similarity.append(similarity)
        similarity_scores.append({aspect_name: score for aspect_name, score in zip(aspect_names, sentence_similarity)})
    return similarity_scores

In [47]:
# review_sentences = df['input_text'][0:100].to_list() --working fine

# review_sentences = df['input_text']

review_sentences = df['input_text']

#for each review in 'cleaned_text' and get a list of aspects

# Compute aspect embeddings
aspect_embeddings = compute_aspect_embeddings(aspects, nlp)

# Compute semantic similarity
semantic_similarity_scores = compute_semantic_similarity(review_sentences, aspect_embeddings)

# Choose the aspect with the highest similarity score for each sentence
chosen_aspects = [max(scores, key=scores.get) for scores in semantic_similarity_scores]

In [48]:
# Output the chosen aspects for each sentence
for sentence, aspect in zip(review_sentences, chosen_aspects):
    print("Sentence:", sentence)
    print("Chosen Aspect:", aspect)
    print()

Streaming output truncated to the last 5000 lines.
Chosen Aspect: food

Sentence: not imagine not rushing eat
Chosen Aspect: food

Sentence: die hard peep lover
Chosen Aspect: ambience

Sentence: live neightborhood regular
Chosen Aspect: service

Sentence: id definitely recommend little place
Chosen Aspect: ambience

Sentence: recommend getting reservation even though saw people seated without one
Chosen Aspect: service

Sentence: tried various slice joints around village far best one
Chosen Aspect: service

Sentence: ive twice
Chosen Aspect: food

Sentence: keep fingers crossed whole subway ride hoping table sit not share rice congee soup people
Chosen Aspect: food

Sentence: glass leaping lizard glass prosecco mussels everything happy
Chosen Aspect: ambience

Sentence: decor nice minimalist food simple yet very well presented cooked wine list matches food very well
Chosen Aspect: food

Sentence: appetizers ate dim sum variety foods impossible criticize food
Chosen Aspect: food

Sente

In [49]:
df['aspect_predicted'] = chosen_aspects

In [50]:
df.head()

,input_text,aspect,polarity,extracted_nouns,aspect_predicted
0,decor nice casual fine dining,ambience,conflict,"['decor', 'dining']",ambience
1,find returning quite often,anecdotesmiscellaneous,positive,[],service
2,seating trying get maximum amount people resta...,ambience,negative,"['people', 'amount', 'neighbor', 'seating']",service
3,like took leftover chicken poured oil sprinkle...,food,negative,"['chicken', 'pepper', 'oil', 'red', 'sauce', '...",food
4,finest latin women youll see life hang jimmys,ambience,positive,"['life', 'women']",ambience


# Accuracy check

In [51]:
df2 = df[df['aspect'] != "anecdotesmiscellaneous"]

In [52]:
cnt = (df2['aspect'] == df2['aspect_predicted']).sum()

In [53]:
cnt

1304

In [54]:
df2.shape

(2035, 5)

In [55]:
print("Accuracy: ", cnt / len(df2))

Accuracy:  0.6407862407862408


# 65% accuracy

# Anurag's Changes

In [56]:
# Anurag's changes
print(df2.head())

                                          input_text    aspect  polarity  \
0                      decor nice casual fine dining  ambience  conflict   
2  seating trying get maximum amount people resta...  ambience  negative   
3  like took leftover chicken poured oil sprinkle...      food  negative   
4      finest latin women youll see life hang jimmys  ambience  positive   
7         ive bunch times service always outstanding   service  positive   

                                     extracted_nouns aspect_predicted  
0                                ['decor', 'dining']         ambience  
2        ['people', 'amount', 'neighbor', 'seating']          service  
3  ['chicken', 'pepper', 'oil', 'red', 'sauce', '...             food  
4                                  ['life', 'women']         ambience  
7                      ['service', 'bunch', 'times']          service  


# Fabian's Changes

In [ ]:
# Fabian's Changes

# Logistic Regression

In [57]:
%load_ext autoreload
%autoreload 2

from models import logistic

logi = logistic.Logistic()
tr = pd.DataFrame(train_dataset)
logi.train(df2)
report = logi.test(test_dataset)
print(report)

              precision    recall  f1-score   support

           0       0.54      0.47      0.50       163
           1       0.69      0.93      0.80       445
           2       0.50      0.01      0.02        97
           3       0.00      0.00      0.00        38

    accuracy                           0.66       743
   macro avg       0.43      0.35      0.33       743
weighted avg       0.60      0.66      0.59       743



# BERT UnCased

In [58]:
%load_ext autoreload
%autoreload 2

from models import bert_uncased

bert_uncase = bert_uncased.BERTUncased(50)
bert_uncase.train(df2)
report = bert_uncase.test(test_dataset)
print(report)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting Epoch 1
Loss:  0.8249273732421675
Starting Epoch 2
Loss:  0.5891124629077076
Starting Epoch 3
Loss:  0.42629437978344664
Starting Epoch 4
Loss:  0.3143389125588791
Starting Epoch 5
Loss:  0.24033655962689043
Starting Epoch 6
Loss:  0.19267101767006348
Starting Epoch 7
Loss:  0.162189969160111
Starting Epoch 8
Loss:  0.1420569194104314
Starting Epoch 9
Loss:  0.1085468361129955
Starting Epoch 10
Loss:  0.10777419238110679
              precision    recall  f1-score   support

    conflict       0.23      0.24      0.23        38
    negative       0.42      0.81      0.55       163
     neutral       0.33      0.08      0.13        97
    positive       0.86      0.71      0.78       445

    accuracy                           0.63       743
   macro avg       0.46      0.46      0.42       743
weighted avg       0.66      0.63      0.62       743



In [ ]:
# Common cell to comapre and contrast varoius above models

In [ ]:
# Load and test a few sentences from the YELP dataset